<style type="text/css">.reveal pre { line-height: 1.3em; }</style>
# Python performance tips and best practice for CLIMADA developers: for-loops, for-loops, for-loops

Thomas Vogt (PIK)

CLIMADA coding convention workshop<br />
January 18, 2021

<h1>Table of Contents (not in the slideshow)<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Summary" data-toc-modified-id="Summary-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Summary</a></span></li><li><span><a href="#Profiling" data-toc-modified-id="Profiling-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Profiling</a></span></li><li><span><a href="#General-considerations" data-toc-modified-id="General-considerations-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>General considerations</a></span><ul class="toc-item"><li><span><a href="#for-loops" data-toc-modified-id="for-loops-3.1"><span class="toc-item-num">3.1&nbsp;&nbsp;</span>for-loops</a></span></li><li><span><a href="#Converting-data-structures" data-toc-modified-id="Converting-data-structures-3.2"><span class="toc-item-num">3.2&nbsp;&nbsp;</span>Converting data structures</a></span></li><li><span><a href="#Always-consider-several-implementations" data-toc-modified-id="Always-consider-several-implementations-3.3"><span class="toc-item-num">3.3&nbsp;&nbsp;</span>Always consider several implementations</a></span></li><li><span><a href="#Efficient-algorithms" data-toc-modified-id="Efficient-algorithms-3.4"><span class="toc-item-num">3.4&nbsp;&nbsp;</span>Efficient algorithms</a></span></li><li><span><a href="#Memory-usage" data-toc-modified-id="Memory-usage-3.5"><span class="toc-item-num">3.5&nbsp;&nbsp;</span>Memory usage</a></span></li></ul></li><li><span><a href="#NumPy-related-tips-and-best-practice" data-toc-modified-id="NumPy-related-tips-and-best-practice-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>NumPy-related tips and best practice</a></span><ul class="toc-item"><li><span><a href="#Vectorized-functions" data-toc-modified-id="Vectorized-functions-4.1"><span class="toc-item-num">4.1&nbsp;&nbsp;</span>Vectorized functions</a></span></li><li><span><a href="#Broadcasting" data-toc-modified-id="Broadcasting-4.2"><span class="toc-item-num">4.2&nbsp;&nbsp;</span>Broadcasting</a></span></li><li><span><a href="#A-note-on-in-place-operations" data-toc-modified-id="A-note-on-in-place-operations-4.3"><span class="toc-item-num">4.3&nbsp;&nbsp;</span>A note on in-place operations</a></span></li></ul></li><li><span><a href="#Sparse-matrices" data-toc-modified-id="Sparse-matrices-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Sparse matrices</a></span></li><li><span><a href="#Fast-for-loops-using-Numba" data-toc-modified-id="Fast-for-loops-using-Numba-6"><span class="toc-item-num">6&nbsp;&nbsp;</span>Fast for-loops using Numba</a></span></li><li><span><a href="#Parallelizing-tasks" data-toc-modified-id="Parallelizing-tasks-7"><span class="toc-item-num">7&nbsp;&nbsp;</span>Parallelizing tasks</a></span></li><li><span><a href="#Read-NetCDF-datasets-with-xarray" data-toc-modified-id="Read-NetCDF-datasets-with-xarray-8"><span class="toc-item-num">8&nbsp;&nbsp;</span>Read NetCDF datasets with <code>xarray</code></a></span></li><li><span><a href="#Take-home-messages" data-toc-modified-id="Take-home-messages-9"><span class="toc-item-num">9&nbsp;&nbsp;</span>Take-home messages</a></span></li></ul></div>

## Summary
In this guide, we will cover these central points:

⏲️ **Use profiling tools** to find and assess performance bottlenecks.<br />
🔁 **Replace for-loops** by built-in functions and efficient external implementations.<br />
📝 **Consider algorithmic performance**, not only implementation performance.<br />
🧊 **Get familiar with NumPy:** vectorized functions, slicing, masks and broadcasting.<br />
⚫ **Miscellaneous:** sparse arrays, Numba, parallelization, huge files (xarray), memory.<br />
⚠️ **Don't over-optimize** at the expense of readability and usability.

## Profiling
Python comes with powerful packages for the **performance assessment** of your code. Within IPython and notebooks, there are several magic commands for this task:

* `%time`: Time the execution of a single statement
* `%timeit`: Time repeated execution of a single statement for more accuracy
* `%%timeit` Does the same as `%timeit` for a whole cell
* `%prun`: Run code with the profiler
* `%lprun`: Run code with the line-by-line profiler
* `%memit`: Measure the memory use of a single statement
* `%mprun`: Run code with the line-by-line memory profiler

More information on profiling in the [Python Data Science Handbook](https://jakevdp.github.io/PythonDataScienceHandbook/01.07-timing-and-profiling.html).

Also useful: unofficial Jupyter extension [Execute Time](https://jupyter-contrib-nbextensions.readthedocs.io/en/latest/nbextensions/execute_time/readme.html).

While it's easy to assess how fast or slow parts of your code are, including finding the bottlenecks, **generating an improved version of it is much harder**. This guide is about **simple best practices** that everyone should know who works with Python, especially when models are performance-critical.

In the following, we will **focus on arithmetic operations** because they play an important role in CLIMADA. Operations on non-numeric objects like strings, graphs, databases, file or network IO might be just as relevant inside and outside of the CLIMADA context. Some of the tips presented here do also apply to other contexts, but **it's always worth looking for context-specific performance guides**.

## General considerations
This section will be concerned with:

🔁 **for-loops** and built-ins<br />
📦 **external implementations** and converting data structures<br />
📝 **algorithmic efficiency**<br />
💾 **memory usage**

𝚺 As this section's toy example, let's assume we want to sum up all the numbers in a list:

In [1]:
list_of_numbers = list(range(10000))

### for-loops

A developer with a background in C++ would probably loop over the entries of the list:

In [2]:
%%timeit
result = 0
for i in list_of_numbers:
    result += i

239 µs ± 3.68 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


The built-in function `sum` is much faster:

In [3]:
%timeit sum(list_of_numbers)

47.7 µs ± 3.56 µs per loop (mean ± std. dev. of 7 runs, 10000 loops each)


The timing improves by a factor of 5-6 and this is not a coincidence: **for-loops generally tend to get prohibitively expensive** when the number of iterations increases.

**💡 When you have a for-loop with many iterations in your code, check for built-in functions or efficient external implementations of your programming task.**

A special case worth noting are `append` operations on lists which can often be replaced by more efficient *list comprehensions*.

### Converting data structures

💡 **When you find an external library that solves your task efficiently, always consider that it might be necessary to convert your data structure which takes time.**

For arithmetic operations, NumPy is a great library, but if your data comes as a Python list, NumPy will spend quite some time converting it to a NumPy array:

In [4]:
import numpy as np
%timeit np.sum(list_of_numbers)

489 µs ± 42.3 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


This operation is even slower than the for-loop!

However, if you can somehow obtain your data in the form of **NumPy arrays from the start**, or if you perform many operations that might compensate for the conversion time, the gain in performance can be considerable:

In [5]:
# do the conversion outside of the `%timeit`
ndarray_of_numbers = np.array(list_of_numbers)
%timeit np.sum(ndarray_of_numbers)

7.58 µs ± 336 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)


Indeed, this is 5-6 times faster than the built-in `sum` and 20-30 times faster than the for-loop.

### Always consider several implementations

Even for such a basic task as summing, there exist several implementations whose performance can vary more than you might expect:

In [6]:
%timeit ndarray_of_numbers.sum()
%timeit np.einsum("i->", ndarray_of_numbers)

6.1 µs ± 231 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)
4.55 µs ± 203 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)


This is up to 50 times faster than the for-loop. More information about the `einsum` function will be given in the NumPy section of this guide.

### Efficient algorithms

💡 **Consider algorithmic performance, not only implementation performance.**

All of the examples above do exactly the same thing, algorithmically. However, often the largest performance improvements can be obtained from **algorithmical changes**. This is the case when your model or your data contain symmetries or more complex structure that allows you to skip or boil down arithmetic operations.

In our example, we are summing the numbers from 1 to 10,000 and it's a well known mathematical theorem that this can be done using only two multiplications and an increment:

In [7]:
n = max(list_of_numbers)
%timeit 0.5 * n * (n + 1)

65.5 ns ± 0.705 ns per loop (mean ± std. dev. of 7 runs, 10000000 loops each)


Not surprisingly, This is almost 100 times faster than even the fastest implementation of the 10,000 summing operations listed above.

You don't need a degree in maths to find algorithmical improvements. Other algorithmical improvements that are often easy to detect are:
* **Filter your data set as much as possible** to perform operations only on those entries that are really relevant. <br> ***Example:*** When computing a physical hazard (e.g. extreme wind) with CLIMADA, restrict to Centroids on land unless you know that some of your exposure is off shore.
* Make sure to **detect inconsistent or trivial input parameters early on**, before starting any operations. <br> ***Example:*** If your code does some complicated stuff and applies a user-provided normalization factor at the very end, make sure to check that the factor is not 0 before you start applying those complicated operations.

📝 **In general: Before starting to code, take pen and paper and write down what you want to do from an algorithmic perspective.**

### Memory usage
💡 **Be careful with deep copies of large data sets and only load portions of large files into memory as needed.**

Write your code in such a way that you **handle large amounts of data chunk by chunk** so that Python does not need to load everything into memory before performing any operations. When you do, Python's [generators](https://wiki.python.org/moin/Generators) might help you with the implementation.

🐌 Allocating unnecessary amounts of memory might slow down your code substantially due to swapping.

## NumPy-related tips and best practice

As mentioned above, arithmetic operations in Python can profit a lot from NumPy's capabilities. In this section, we collect some tips how to make use of NumPy's capabilities when performance is an issue.

### Vectorized functions
We mentioned above that Python's **for-loops are really slow**. This is even more important when looping over the entries in a NumPy array. Fortunately, NumPy's masks, slicing notation and vectorization capabilities help to avoid for-loops in almost every possible situation:

In [8]:
# TASK: compute the column-sum of a 2-dimensional array
input_arr = np.random.rand(100, 3)

In [9]:
%%timeit
# SLOW: summing over columns using loops
output = np.zeros(100)
for row_i in range(input_arr.shape[0]):
    for col_i in range(input_arr.shape[1]):
        output[row_i] += input_arr[row_i, col_i]

125 µs ± 371 ns per loop (mean ± std. dev. of 7 runs, 10000 loops each)


In [10]:
# FASTER: using NumPy's vectorized `sum` function with `axis` attribute
%timeit output = input_arr.sum(axis=1)

3.38 µs ± 11.6 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)


In the special case of multiplications and sums (linear operations) over the axes of two multi-dimensional arrays, NumPy's **`einsum`** is even faster:

In [11]:
%timeit output = np.einsum("ij->i", input_arr)

1.98 µs ± 10 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)


Another example: **Euclidean norms**

In [12]:
many_vectors = np.random.rand(1000, 3)
%timeit np.sqrt((many_vectors**2).sum(axis=1))
%timeit np.linalg.norm(many_vectors, axis=1)
%timeit np.sqrt(np.einsum("...j,...j->...", many_vectors, many_vectors))

20 µs ± 696 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)
21.8 µs ± 640 ns per loop (mean ± std. dev. of 7 runs, 10000 loops each)
8.32 µs ± 191 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)


For more information about the capabilities of NumPy's `einsum` function, continue reading here: https://numpy.org/doc/stable/reference/generated/numpy.einsum.html

Not only `sum`, but many NumPy functions come with similar vectorization capabilities. You can take minima, maxima, means or standard deviations along selected axes. But did you know that the same is true for the `diff` and `argmin` functions?

In [13]:
arr = np.random.randint(low=0, high=10, size=(4, 3))
arr

array([[1, 0, 6],
       [1, 4, 4],
       [6, 5, 5],
       [6, 2, 6]])

In [14]:
arr.argmin(axis=1)

array([1, 0, 1, 1])

### Broadcasting

When operations are performed on several arrays, possibly of differing shapes, be sure to use NumPy's **broadcasting** capabilities. This will save you a lot of memory and time when performing arithmetic operations.

***Example:*** We want to multiply the columns of a two-dimensional array by values stored in a one-dimensional array. There are two naive approaches to this:

In [15]:
input_arr = np.random.rand(100, 3)
col_factors = np.random.rand(3)

In [16]:
# SLOW: stack/tile the one-dimensional array to be two-dimensional
%timeit output = np.tile(col_factors, (input_arr.shape[0], 1)) * input_arr

5.29 µs ± 1.45 µs per loop (mean ± std. dev. of 7 runs, 100000 loops each)


In [17]:
%%timeit
# SLOW: loop over columns and factors
output = input_arr.copy()
for i, factor in enumerate(col_factors):
    output[:, i] *= factor

7.78 µs ± 613 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)


The idea of *broadcasting* is that NumPy **automatically matches axes from right to left and implicitly repeats data along missing axes** if necessary:

In [18]:
%timeit output = col_factors * input_arr

1.34 µs ± 77.3 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)


For automatic broadcasting, the *trailing* dimensions of two arrays have to match. NumPy is matching the shapes of the arrays *from right to left*. If you happen to have arrays where other dimensions match, **you have to tell NumPy which dimensions to add by adding an axis of length 1 for each missing dimension**:

In [19]:
input_arr = np.random.rand(3, 100)
row_factors = np.random.rand(3)
output = row_factors.reshape(3, 1) * input_arr

Because this concept is so important, there is a short-hand notation for adding an axis of length 1. In the slicing notation, **add `None` in those positions where broadcasting should take place**.

In [20]:
input_arr = np.random.rand(3, 100)
row_factors = np.random.rand(3)
output = row_factors[:, None] * input_arr

In [21]:
input_arr = np.random.rand(7, 3, 5, 4, 6)
factors = np.random.rand(7, 3, 4)
output = factors[:, :, None, :, None] * input_arr

### A note on in-place operations
While **in-place operations** are generally faster than long and explicit expressions, they shouldn't be over-estimated when looking for performance bottlenecks. Often, the loss in code readability is not justified because NumPy's memory management is really fast.

💡 **Don't over-optimize!**

In [22]:
shape = (1200, 1700)
arr_a = np.random.rand(*shape)
arr_b = np.random.rand(*shape)
arr_c = np.random.rand(*shape)

In [23]:
# long expression in one line
%timeit arr_d = arr_c * (arr_a + arr_b) - arr_a + arr_c

17.2 ms ± 4.32 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [24]:
%%timeit
# almost same performance: in-place operations
arr_d = arr_a + arr_b
arr_d *= arr_c
arr_d -= arr_a
arr_d += arr_c

17.7 ms ± 3.3 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [25]:
%load_ext memory_profiler

In [26]:
# long expression in one line
%memit arr_d = arr_c * (arr_a + arr_b) - arr_a + arr_c

peak memory: 156.96 MiB, increment: 31.21 MiB


In [27]:
%%memit
# almost same memory usage: in-place operations
arr_d = arr_a + arr_b
arr_d *= arr_c
arr_d -= arr_a
arr_d += arr_c

peak memory: 157.14 MiB, increment: 0.00 MiB


## Sparse matrices

In many contexts, we deal with sparse matrices or sparse data structures, i.e. two-dimensional arrays where most of the entries are 0. In CLIMADA, this is especially the case for the `intensity` attributes of `Hazard` objects. This kind of data is usually handled using SciPy's submodule [scipy.sparse](https://docs.scipy.org/doc/scipy/reference/sparse.html).

💡 **When dealing with sparse matrices make sure that you always understand exactly which of your variables are sparse and which are dense and only switch from sparse to dense when absolutely necessary.**

## Fast for-loops using Numba
As a last resort, if there's no way to avoid a for-loop even with NumPy's vectorization capabilities, you can use the `@njit` decorator provided by the Numba package:

In [28]:
from numba import njit

@njit
def sum_array(arr):
    result = 0.0
    for i in range(arr.shape[0]):
        result += arr[i]
    return result

In fact, the Numba function is more than 100 times faster than without the decorator:

In [29]:
input_arr = np.float64(np.random.randint(low=0, high=10, size=(10000,)))

In [30]:
%timeit sum_array(input_arr)

9.92 µs ± 192 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)


In [31]:
# Call the function without the @njit
%timeit sum_array.py_func(input_arr)

1.7 ms ± 85.5 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


However, whenever available, **NumPy's own vectorized functions will usually be faster than Numba**.

In [32]:
%timeit np.sum(input_arr)
%timeit input_arr.sum()
%timeit np.einsum("i->", input_arr)

5.95 µs ± 525 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)
4.66 µs ± 455 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)
6.93 µs ± 396 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)


💡 **Make sure you understand the basic idea behind Numba before using it, read the [Numba docs](https://numba.readthedocs.io/en/stable/user/5minguide.html).**

💡 **Don't use `@jit`, but use `@njit` which is an alias for `@jit(nopython=True)`.**

When you know what you are doing, the `fastmath` and `parallel` options can boost performance even further: read more about this in the [Numba docs](https://numba.pydata.org/numba-doc/dev/user/performance-tips.html).

## Parallelizing tasks

Depending on your hardware setup, parallelizing tasks using [pathos](https://pathos.readthedocs.io/en/latest/pathos.html) and [Numba's automatic parallelization feature](https://numba.readthedocs.io/en/stable/user/parallel.html) can improve the performance of your implementation.

💡 **Expensive hardware is no excuse for inefficient code.**

Many tasks in CLIMADA could profit from GPU implementations. However, currently there are **no plans to include GPU support in CLIMADA** because of the considerable development and maintenance workload that would come with it. If you want to change this, contact the core team of developers, open an issue or mention it in the bi-weekly meetings.

## Read NetCDF datasets with `xarray`
When dealing with NetCDF datasets, memory is often an issue, because even if the file is only a few megabytes in size, the uncompressed raw arrays contained within can be several gigabytes large (especially when data is sparse or similarly structured). One way of dealing with this situation is to open the dataset with `xarray`.

💡 **`xarray` allows to read the shape and type of variables contained in the dataset without loading any of the actual data into memory.**

Furthermore, when loading slices and arithmetically aggregating variables, memory is allocated not more than necessary, but values are obtained on-the-fly from the file.

## Take-home messages
⏲️ **Use profiling tools** to find and assess performance bottlenecks.<br />
🔁 **Replace for-loops** by built-in functions and efficient external implementations.<br />
📝 **Consider algorithmic performance**, not only implementation performance.<br />
🧊 **Get familiar with NumPy:** vectorized functions, slicing, masks and broadcasting.<br />
⚫ **Miscellaneous:** sparse arrays, Numba, parallelization, huge files (xarray), memory.<br />
⚠️ **Don't over-optimize** at the expense of readability and usability.